In [1]:
import pandas as pd
import pickle
from gensim.models.phrases import Phrases, Phraser
from collections import defaultdict  # For word frequency

In [2]:
with open("df_clean.csv", "rb") as fp:   # Unpickling
    df_clean = pickle.load(fp)

In [3]:
df_clean = df_clean[~df_clean.clean.str.contains("gt")]

In [4]:
sent = [row.split() for row in df_clean['clean']]

In [5]:
phrases = Phrases(sent, min_count=3 0)

In [6]:
bigram = Phraser(phrases)
sentences = bigram[sent]

In [7]:
import multiprocessing

from gensim.models import Word2Vec

In [8]:
cores = multiprocessing.cpu_count() # Count the number of cores in a computer

In [18]:
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [19]:
w2v_model.build_vocab(sentences[:100000])

### Google Model

In [11]:
from gensim.models import KeyedVectors
filename = 'GoogleNews-vectors-negative300.bin'
model = KeyedVectors.load_word2vec_format(filename, binary=True)

In [16]:
model.most_similar(positive=["man"])

[('woman', 0.7664012312889099),
 ('boy', 0.6824870705604553),
 ('teenager', 0.6586930155754089),
 ('teenage_girl', 0.6147903800010681),
 ('girl', 0.5921714305877686),
 ('suspected_purse_snatcher', 0.571636438369751),
 ('robber', 0.5585119724273682),
 ('Robbery_suspect', 0.5584410429000854),
 ('teen_ager', 0.5549196600914001),
 ('men', 0.5489762425422668)]

### Fine Tunning